In [1]:
#Diretório dos dados.
import os
data_dir = 'TCC - Data'

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import numpy as np
from matplotlib import pyplot as plt

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#Lista de pastas no diretório de dados
os.listdir(data_dir)
#Lista todos os arquivos armazenados na pasta 'Healthy'
#os.listdir(os.path.join(data_dir, 'Healthy'))

class_labels = ['Bacterial Blight', 'Blast', 'Brownspot', 'Healthy', 'Tungro']

In [4]:
data = tf.keras.utils.image_dataset_from_directory(
    data_dir, label_mode='categorical',image_size=(300,300))

Found 7123 files belonging to 5 classes.


In [5]:
def one_hot_decode(predicted):
    decode = tf.argmax(predicted).numpy()
    return class_labels[decode]
    


In [6]:
norm_data = data.map(lambda x, y: (x/255, y))

In [7]:
#Total lenght of data batches = 223
#Total batche used = 222
len(data)
# 70% - Treino - 156
train_size = int(len(norm_data)*.7)
# 20% - Validação - 44
validation_size = int(len(norm_data)*.2)
# 10% - Teste - 22 (+1)
test_size = int(len(norm_data)*.1)+1

In [8]:
#Separating the data into 3 variables, each containing a number of batches and each batch containing 32 images.
train = norm_data.take(train_size)
validation = norm_data.skip(train_size).take(validation_size)
test = norm_data.skip(train_size+validation_size).take(test_size)

In [20]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(300, 300, 3),
    pooling="max",
    classifier_activation="softmax",
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(300, 300, 3))
x = (inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer


# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
#x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = tf.keras.layers.Dense(5)(x)
layer = tf.keras.layers.Softmax(5)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 300, 300, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 14,717,253
Trainable params: 2,565
Non-trainable params: 14,714,688
_________________________________________________________________


In [21]:
model.compile('adam', loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

epochs = 20
hist = model.fit(train, epochs=epochs, validation_data=validation)

Epoch 1/20
 13/156 [=>............................] - ETA: 46s - loss: 8.4062 - accuracy: 0.1514

KeyboardInterrupt: 

In [ ]:
base_model.trainable = True
model.summary()

In [ ]:
model.compile('adam', loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

epochs = 10
hist = model.fit(train, epochs=epochs, validation_data=validation)